In [1]:
# Colabis kasutamiseks
from google.colab import drive
#drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

In [2]:
!pip install estnltk==1.7.2
!pip install evaluate
!pip install seqeval
!pip install nervaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Succ

In [15]:
# Impordid
import os
import numpy as np
import json
import evaluate
from collections import defaultdict
from datasets import Dataset, DatasetDict
from itertools import product
from datetime import datetime

# _
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from transformers import TFAutoModelForTokenClassification, create_optimizer, AdamWeightDecay
from transformers.keras_callbacks import KerasMetricCallback

# data_processing.py samas kaustas
from data_processing import (
    TAG2IDX, IDX2TAG, ALL_TAGS,
    get_dataset_paths, preprocess, split_to_token_and_tag,
    transform_set, process_all,
    save_split_to_json, save_dataset_to_json, load_split_from_json, load_dataset_from_json,
    load_and_process_dataset, combine_datasetdicts, load_all)

In [5]:
ewt_dataset, edt_dataset, combined_dataset = load_all(from_json=True)
# sama, mis üleval
#ewt_from_json = load_dataset_from_json('ewt')
#edt_from_json = load_dataset_from_json('edt')
#combined_from_json = load_dataset_from_json()

# nullist
#ewt_dataset, edt_dataset, combined_dataset = load_all(from_json=False)
#ewt_dataset = load_and_process_dataset('ewt')
#edt_dataset = load_and_process_dataset('edt')
#combined_dataset = combine_datasetdicts(ewt_dataset, edt_dataset)

In [6]:
#tokenizer = AutoTokenizer.from_pretrained("tartuNLP/EstBERT") #, max_length=128, padding="max_length", truncation=True
tokenizer = AutoTokenizer.from_pretrained("tartuNLP/EstBERT", max_length=128, padding="max_length", truncation="longest_first")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/410k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
# https://huggingface.co/docs/transformers/en/tasks/token_classification

def tokenize_and_align_labels(sent, label_all=True):
  # Sisend: üks lause
  # Väljund: tokenizeri abil sõnestatud lause, sõnastik sisaldab input_ids, token_type_ids, attention_mask ja labels
  # input_ids on lause 'arvulisel' kujul, igale ID-le vastab mingi sõna (teisendamine funktsiooni tokenizer.convert_ids_to_tokens(input_ids) abil)
  # labels on märgendite list, kus märgendid kattuvad tokenizeri abil sõnestatud lausega

  #tokenized_inputs = tokenizer(sent['tokens'], is_split_into_words=True) #, truncation=True, is_split_into_words=True, max_length=128, padding="max_length"
  tokenized_inputs = tokenizer(sent['tokens'], is_split_into_words=True, max_length=128, padding="max_length", truncation="longest_first")
  labels = []
  word_ids = tokenized_inputs.word_ids()
  prev_word = None

  for word in word_ids:
    if word is None:
      label = -100
    elif word != prev_word:
      label = sent['tags'][word]
    else:
      label = sent['tags'][word] if label_all else -100
      #label = -100 # reetpa67 -> B-Per; 'reet', '##pa', '##67' -> 11, -100, -100 ehk B-Per
    labels.append(label)
    prev_word = word

  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [10]:
def print_aligned_example(dataset, split='dev', i=0):
  example = dataset[split][i]
  print(f'Lause algselt: {example}')

  aligned_example = tokenize_and_align_labels(example, True)
  tokens = tokenizer.convert_ids_to_tokens(aligned_example.input_ids)
  print(f'Lause nö arvulisel kujul: {aligned_example.input_ids}')
  print(f'Sõnestatud: {tokens}')
  aligned_labels = aligned_example['labels']
  print(f'Märgendid arvulisel kujul paigas: {aligned_labels}')
  aligned_labels_text = [ALL_TAGS[label] for label in aligned_labels if label>=0]
  print(f'Märgendid: {aligned_labels_text}')

print_aligned_example(ewt_dataset, 'train', 1)

Lause algselt: {'id': 1, 'tags': [11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'tokens': ['reetpa67', ':', 'Juba', 'teist', 'aastat', 'ei', 'õitse', 'mul', 'aias', 'nartsissid', '.']}
Lause nö arvulisel kujul: [2, 21999, 199, 23498, 137, 368, 1325, 887, 82, 6917, 164, 392, 10013, 3553, 3188, 379, 15, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sõnestatud: ['[CLS]', 'reet', '##pa', '##67', ':', 'juba', 'teist', 'aastat', 'ei', 'oi', '##tse', 'mul', 'aias', 'nar', '##tsis', '##sid', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [ ]:
# NB! Kasutades DatasetDict ja Dataset on see siin ebavajalik, saab otse dataset.map() kasutada
def tokenize_dataset(dataset_dict):
    # Sisend: andmestik sõnastikuna, mis sisaldab train/dev/test hulkasid
    # Väljund:
    processed_dataset = {}

    for split_name, split_data in dataset_dict.items():
        #examples = [split_data[idx] for idx in range(len(split_data))]
        #print(examples == split_data)
        #tokenized_dataset = list(map(tokenize_and_align_labels, examples))
        tokenized_dataset = list(map(tokenize_and_align_labels, split_data))
        processed_split = {
            'input_ids': [],
            'attention_mask': [],
            'labels': []
        }

        for item in tokenized_dataset:
            processed_split['input_ids'].append(item['input_ids'])
            processed_split['attention_mask'].append(item['attention_mask'])
            processed_split['labels'].append(item['labels'])

        processed_dataset[split_name] = processed_split

    return processed_dataset

In [11]:
tokenized_ewt = ewt_dataset.map(tokenize_and_align_labels)
#tokenized_edt = edt_from_json.map(tokenize_and_align_labels)
#tokenized_combined = combined_from_json.map(tokenize_and_align_labels)

Map:   0%|          | 0/5444 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/913 [00:00<?, ? examples/s]

In [12]:
print(tokenized_ewt['train'])

Dataset({
    features: ['id', 'tags', 'tokens', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 5444
})


In [13]:
#data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")
data_collator_pt = DataCollatorForTokenClassification(tokenizer=tokenizer)
seqeval = evaluate.load("seqeval")

In [14]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [ALL_TAGS[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [ALL_TAGS[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# tensorflow variant
def finetune_model(tokenized_dataset, epochs=3, batch_size=8):
  num_train_steps = (len(tokenized_dataset["train"]) // batch_size) * epochs
  optimizer, lr_schedule = create_optimizer(
      init_lr=1e-5,
      num_train_steps=num_train_steps,
      weight_decay_rate=0.01,
      num_warmup_steps=0,
      adam_beta1=0.9,
      adam_beta2=0.98,
      adam_epsilon=1e-6
  )

  model = TFAutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT", num_labels=len(ALL_TAGS), id2label=IDX2TAG, label2id=TAG2IDX)

  tf_train_set = model.prepare_tf_dataset(
      tokenized_dataset['train'],
      shuffle=True,
      batch_size=batch_size,
      collate_fn=data_collator,
  )
  tf_validation_set = model.prepare_tf_dataset(
      tokenized_dataset['dev'],
      shuffle=False,
      batch_size=batch_size,
      collate_fn=data_collator,
  )

  model.compile(optimizer=optimizer)

  metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
  callbacks = [metric_callback]

  model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=epochs, callbacks=callbacks)

  return model

#combined_model = finetune_model(tokenized_combined)
#Epoch 3/3
#3755/3755 [==============================] - 619s 165ms/step - loss: 0.0727 - val_loss: 0.1575 - precision: 0.6984 - recall: 0.6915 - f1: 0.6949 - accuracy: 0.9617

In [ ]:
def finetune_model_pytorch(tokenized_dataset, epochs=3, batch_size=16, lr=5e-5, freeze=False):

  model = AutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT", num_labels=len(ALL_TAGS), id2label=IDX2TAG, label2id=TAG2IDX)

  if freeze:
    for param in model.bert.embeddings.parameters():
      param.requires_grad = False
    for layer in model.bert.encoder.layer[:3]:
      for param in layer.parameters():
        param.requires_grad = False

  training_args = TrainingArguments(
      report_to='none',
      output_dir='./results',
      learning_rate=lr,
      lr_scheduler_type='polynomial', # default on linear, aga polynomial annab veidi parema tulemuse
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=epochs,
      weight_decay=0.0,
      eval_strategy="epoch",
      save_strategy="epoch",
      optim="adamw_torch",
      load_best_model_at_end=True,
      metric_for_best_model='f1',
      adam_beta1=0.9,
      adam_beta2=0.999,
      adam_epsilon=1e-8,
      fp16=True
  )

  early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=5,
        early_stopping_threshold=0.0001
    )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset['train'],
      eval_dataset=tokenized_dataset['dev'],
      processing_class=tokenizer,
      data_collator=data_collator_pt,
      compute_metrics=compute_metrics,
      callbacks=[early_stopping_callback]
  )

  trainer.train()

  return model, trainer

In [ ]:
def grid_search_pt(tokenized_dataset=None, param_grid=None):
  results = []
  best_f1 = 0
  best_model = None
  best_params = None

  if param_grid:
    param_combinations = [dict(zip(param_grid.keys(), v))
                         for v in product(*param_grid.values())]

  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

  for i, params in enumerate(param_combinations, 1):
    print(f"\n{i}/{len(param_combinations)}")
    print(f"Params: {params}")

    try:
      model = AutoModelForTokenClassification.from_pretrained("tartuNLP/EstBERT", num_labels=len(ALL_TAGS), id2label=IDX2TAG, label2id=TAG2IDX)

      training_args = TrainingArguments(
          report_to='none', # kui seda ei ole, siis kasutab weights&biases, vaja api võtit
          output_dir=f'./results_{timestamp}_{i}',
          learning_rate=params['learning_rate'],
          per_device_train_batch_size=params['batch_size'],
          per_device_eval_batch_size=params['batch_size'],
          num_train_epochs=params['num_train_epochs'],
          weight_decay=params['weight_decay'],
          eval_strategy="epoch",
          save_strategy="epoch",
          optim="adamw_torch",
          load_best_model_at_end=True,
          metric_for_best_model='f1',
          adam_beta1=params['adam_beta1'],
          adam_beta2=params['adam_beta2'],
          adam_epsilon=params['adam_epsilon'],
          fp16=True
      )

      early_stopping_callback = EarlyStoppingCallback(
          early_stopping_patience=2,
          early_stopping_threshold=0.0001
      )

      trainer = Trainer(
          model=model,
          args=training_args,
          train_dataset=tokenized_dataset['train'],
          eval_dataset=tokenized_dataset['dev'],
          processing_class=tokenizer,
          data_collator=data_collator_pt,
          compute_metrics=compute_metrics,
          callbacks=[early_stopping_callback]
      )

      train_result = trainer.train()
      eval_result = trainer.evaluate()

      trial_results = {
          'parameters': params,
          'eval_metrics': eval_result,
          'train_metrics': {
              'train_runtime': train_result.metrics['train_runtime'],
              'train_samples_per_second': train_result.metrics['train_samples_per_second']
          }
      }
      results.append(trial_results)

      if eval_result['eval_f1'] > best_f1:
        best_f1 = eval_result['eval_f1']
        best_model = model
        best_params = params

    except Exception as e:
      print(f"Error {i}: {e}")
      continue

  print(f"parim f1: {best_f1}")
  print(f"parameetrid: {best_params}")

  return best_model, best_params, results

In [ ]:
param_grid = {
    'learning_rate': [1e-5, 5e-5],
    'batch_size': [16],
    'num_train_epochs': [5],
    'weight_decay': [0.01, 0.0],
    'adam_beta1': [0.9],
    'adam_beta2': [0.98, 0.99],
    'adam_epsilon': [1e-6, 1e-8]
}

best_model, best_params, all_results = grid_search_pt(tokenized_dataset=tokenized_ewt, param_grid=param_grid)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



1/16
Params: {'learning_rate': 1e-05, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.01, 'adam_beta1': 0.9, 'adam_beta2': 0.98, 'adam_epsilon': 1e-06}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.316813,0.750249,0.661984,0.703358,0.940634
2,0.305600,0.317966,0.720680,0.706760,0.713652,0.943635
3,0.125900,0.326630,0.729828,0.706760,0.718109,0.945392
4,0.125900,0.327276,0.729754,0.727831,0.728791,0.946636
5,0.099100,0.330332,0.725284,0.727831,0.726556,0.946636


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.280998,0.799603,0.707638,0.750815,0.949052
2,0.169700,0.328181,0.776230,0.733977,0.754513,0.950662
3,0.045500,0.361099,0.767593,0.727831,0.747183,0.951321
4,0.045500,0.384846,0.739280,0.756804,0.747939,0.950223


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

parim f1: 0.7658998646820027
parameetrid: {'learning_rate': 5e-05, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.0, 'adam_beta1': 0.9, 'adam_beta2': 0.98, 'adam_epsilon': 1e-06}


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# parim f1: 0.7658998646820027
# parameetrid: {'learning_rate': 5e-05, 'batch_size': 16, 'num_train_epochs': 5, 'weight_decay': 0.0, 'adam_beta1': 0.9, 'adam_beta2': 0.98, 'adam_epsilon': 1e-06}

ewt_mudel, trainer_m = finetune_model_pytorch(tokenized_ewt, 5, 16, 5e-5)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.281832,0.818465,0.692713,0.750357,0.948247
2,0.173300,0.330366,0.788499,0.710272,0.747344,0.948686
3,0.047600,0.358526,0.775547,0.746269,0.760626,0.951760
4,0.047600,0.387028,0.785714,0.753292,0.769162,0.952346
5,0.018100,0.408722,0.778890,0.751536,0.764969,0.951980


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

In [ ]:
ewt_testimiseks, trainer_t = finetune_model_pytorch(tokenized_ewt, 5, 16, 5e-5)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.217636,0.734644,0.674944,0.703529,0.960804
2,0.111600,0.283895,0.742092,0.688488,0.714286,0.962704
3,0.027700,0.322581,0.704492,0.672686,0.688222,0.961004
4,0.027700,0.349729,0.745192,0.699774,0.721769,0.962504
5,0.009700,0.362110,0.748184,0.697517,0.721963,0.963004


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

In [ ]:
from transformers import pipeline

ewt_ner = pipeline("ner", model=ewt_testimiseks, tokenizer=tokenizer)
s = tokenized_ewt['test'][2]['tokens']
actual = tokenized_ewt['test'][2]['labels']
actual_text = [ALL_TAGS[label] for label in actual if label>=0]
print(actual_text)
print(s)
pred = ewt_ner(s)
print(pred)

Device set to use cuda:0


['B-Per', 'I-Per', 'O', 'B-Loc', 'O', 'O', 'O', 'O', 'B-Eve', 'O', 'O', 'B-Per', 'I-Per', 'O', 'O', 'O', 'O']
['Jürgen', 'Zopp', 'kaotas', 'Petange’is', '64000', 'euro', 'suuruse', 'auhinnafondiga', 'Challenger-turniiri', 'poolfinaalis', 'prantslasele', 'Paul-Henri', 'Mathieu’le', '6:7', ',', '2:6', '.']
[[{'entity': 'B-Per', 'score': 0.99676657, 'index': 1, 'word': 'ju', 'start': 0, 'end': 2}, {'entity': 'B-Per', 'score': 0.5130056, 'index': 2, 'word': '##rgen', 'start': 2, 'end': 6}], [{'entity': 'B-Per', 'score': 0.99400735, 'index': 1, 'word': 'zo', 'start': 0, 'end': 2}, {'entity': 'I-Per', 'score': 0.892888, 'index': 2, 'word': '##pp', 'start': 2, 'end': 4}], [], [{'entity': 'B-Org', 'score': 0.69498736, 'index': 1, 'word': 'peta', 'start': 0, 'end': 4}, {'entity': 'I-Org', 'score': 0.27533016, 'index': 2, 'word': '##nge', 'start': 4, 'end': 7}], [], [], [], [], [{'entity': 'B-Prod', 'score': 0.53563005, 'index': 1, 'word': 'cha', 'start': 0, 'end': 3}, {'entity': 'I-Prod', 'scor